In [4]:
import sys
sys.path.append("../../")
import sklearn
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import load_digits
from deep_logic.utils.base import set_seed
from deep_logic.utils.metrics import Accuracy
from deep_logic.models.relunn import XReluClassifier

set_seed(0)

X, y = load_digits(return_X_y=True)

enc = OneHotEncoder()
y1h = enc.fit_transform(y.reshape(-1, 1)).toarray()
print(f'Before: {y.shape}\nAfter: {y1h.shape}')

n_classes = 1
n_concepts = 10
y2 = np.zeros((len(y), n_classes))
for i, yi in enumerate(y):
    if yi % 2:
        y2[i, 0] = 1
    # else:
    #     y2[i, 0] = 1

X = torch.FloatTensor(y1h)
y = torch.FloatTensor(y2)

dataset = torch.utils.data.TensorDataset(X, y)

loss = torch.nn.BCELoss()
metric = Accuracy()
model = XReluClassifier(n_classes=n_classes, n_features=n_concepts, hidden_neurons=[20, 10, 5], 
                        loss=loss, l1_weight=0.0001)
results = model.fit(dataset, dataset, batch_size=64, epochs=5, l_r=0.01, metric=metric)


Before: (1797,)
After: (1797, 10)


In [18]:
x_sample = X[80,:]
reduced_model = model.get_reduced_model(x_sample)
assert isinstance(reduced_model, torch.nn.Sequential)

explanation = model.explain(x_sample, k=6)
print(x_sample)
print(y[80])
print(explanation)
[*reduced_model.children()][0].weight

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0.])
f1 & ~f3 & ~f5 & ~f7 & ~f9


Parameter containing:
tensor([[-3.3881,  6.6507, -3.9406,  6.3383, -3.5984,  6.5526, -4.4396,  6.3644,
         -4.4089,  6.3198],
        [ 3.6350, -7.0997,  4.2248, -6.7722,  3.8601, -6.9998,  4.7581, -6.7990,
          4.7243, -6.7496]], requires_grad=True)

In [21]:
x_sample = X
explanation = model.explain(x_sample, y[:, 1], k=6, local=False)
print(x_sample)
print(y[80])
print(explanation)
[*reduced_model.children()][0].weight

RuntimeError: Boolean value of Tensor with more than one value is ambiguous